# Interpreter

> ❗❗❗ FIXME: the *parser* not work.

The Interpreter pattern is a design pattern that is used to define a grammatical representation for a language and provide an interpreter to deal with this grammar. It is particularly useful for designing languages, scripting engines, and compilers. Here's a comprehensive example of the Interpreter pattern implemented in Python.

# Example: Arithmetic Expression Interpreter

An interpreter for a simple arithmetic language that supports addition, subtraction, multiplication, and division.

## Step 1: Define the Grammar

The grammar for our language can be defined using the following rules:

- `Expression = Term { ("+" | "-") Term }`
- `Term = Factor { ("*" | "/") Factor }`
- `Factor = Number | "(" Expression ")"`

In [2]:
# Abstract Syntax Tree (AST)
from abc import ABC, abstractmethod

# Abstract Expression
class Expression(ABC):
    @abstractmethod
    def interpret(self) -> float:
        pass

# Terminal Expression for numbers   
class Number(Expression):
    def __init__(self, value: float):
        self.value = value

    def interpret(self) -> float:
        return self.value

# Nonterminal Expression for addition
class Add(Expression):
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right

    def interpret(self) -> float:
        return self.left.interpret() + self.right.interpret()

# Nonterminal Expression for subtraction
class Subtract(Expression):
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right

    def interpret(self) -> float:
        return self.left.interpret() - self.right.interpret()

# Nonterminal Expression for multiplication
class Multiply(Expression):
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right

    def interpret(self) -> float:
        return self.left.interpret() * self.right.interpret()

# Nonterminal Expression for division
class Divide(Expression):
    def __init__(self, left: Expression, right: Expression):
        self.left = left
        self.right = right

    def interpret(self) -> float:
        return self.left.interpret() / self.right.interpret()

In [ ]:
import re

# Parser class to build the expression tree from a string
class Parser:
    def __init__(self, expression: str):
        # Tokenize the input expression
        self.tokens = re.findall(r'\d+|\+|\-|\*|\/|\(|\)', expression)
        self.current_token_index = 0

    def parse(self) -> Expression:
        return self.parse_expression()

    def parse_expression(self) -> Expression:
        left = self.parse_term()
        while self.current_token() in ('+', '-'):
            operator = self.current_token()
            self.consume_token()
            right = self.parse_term()
            if operator == '+':
                left = Add(left, right)
            elif operator == '-':
                left = Subtract(left, right)
        return left

    def parse_term(self) -> Expression:
        left = self.parse_factor()
        while self.current_token() in ('*', '/'):
            operator = self.current_token()
            self.consume_token()
            right = self.parse_factor()
            if operator == '*':
                left = Multiply(left, right)
            elif operator == '/':
                left = Divide(left, right)
        return left

    def parse_factor(self) -> Expression:
        token = self.current_token()
        if token.isdigit():
            self.consume_token()
            return Number(float(token))
        elif token == '(':
            self.consume_token()
            expression = self.parse_expression()
            self.consume_token()  # consume ')'
            return expression
        raise ValueError(f"Unexpected token: {token}")

    def current_token(self) -> str:
        if self.current_token_index < len(self.tokens):
            # NOTE: This is the fix to the code
            return self.tokens[self.current_token_index]
        return ''

    def consume_token(self):
        self.current_token_index += 1

# Main part to test the interpreter
expression = "3 + 5 * ( 10 - 4 ) / 2"
parser = Parser(expression)
print("Tokens:", parser.tokens) # DEBUG
syntax_tree = parser.parse()
result = syntax_tree.interpret()
print(f"The result of '{expression}' is {result}")

Tokens: ['3', '+', '5', '*', '(', '10', '-', '4', ')', '/', '2']
The result of '3 + 5 * ( 10 - 4 ) / 2' is 18.0


## Not Working

The code below does not work due to the lack of a check condition in `consume_token` that avoids going out of the boundaries of the tokens array.

In [ ]:
# Parser # FIXME: Not Working
import re

class Parser:
    def __init__(self, expression: str):
        self.tokens = re.findall(r'\d+|[-+*/()]', expression)
        self.current_token_index = 0

    def parse(self) -> Expression:
        return self.parse_expression()

    def parse_expression(self) -> Expression:
        left = self.parse_term()
        while self.current_token() in ('+', '-'):
            print("Cur Token:", self.current_token_index, self.current_token())  # DEBUG
            operator = self.current_token()
            self.consume_token()
            right = self.parse_term()
            if operator == '+':
                left = Add(left, right)
            elif operator == '-':
                left = Subtract(left, right)
        return left

    def parse_term(self) -> Expression:
        left = self.parse_factor()
        while self.current_token() in ('*', '/'):
            operator = self.current_token()
            self.consume_token()
            right = self.parse_factor()
            if operator == '*':
                left = Multiply(left, right)
            elif operator == '/':
                left = Divide(left, right)
        return left

    def parse_factor(self) -> Expression:
        token = self.current_token()
        if token.isdigit():
            self.consume_token()
            return Number(float(token))
        elif token == '(':
            self.consume_token()
            expression = self.parse_expression()
            self.consume_token()  # consume ')'
            return expression
        raise ValueError(f"Unexpected token: {token}")

    def current_token(self) -> str:
        print("  ", self.current_token_index) # DEBUG
        # FIXME: add a boundary-check condition
        return self.tokens[self.current_token_index]

    def consume_token(self):
        self.current_token_index += 1

# Client

expression = "3 + 5 * ( 10 - 4 ) / 2"
parser = Parser(expression)
print("Tokens:", parser.tokens) # DEBUG
syntax_tree = parser.parse()    

result = syntax_tree.interpret()
print(f"The result of '{expression}' is {result}")

Tokens: ['3', '+', '5', '*', '(', '10', '-', '4', ')', '/', '2']
   0
   1
   1
   1
Cur Token: 1 +
   1
   2
   3
   3
   4
   5
   6
   6
   6
Cur Token: 6 -
   6
   7
   8
   8
   9
   9
   10
   11
   11
The result of '3 + 5 * ( 10 - 4 ) / 2' is 18.0


In [12]:
# Client

expression = "3 + 5 * ( 10 - 4 ) / 2"
parser = Parser(expression)
syntax_tree = parser.parse()    
result = syntax_tree.interpret()
print(f"The result of '{expression}' is {result}")

   0
   1
   1
   1
1 +
   1
   2
   3
   3
   4
   5
   6
   6
   6
6 -
   6
   7
   8
   8
   9
   9
   10
   11


IndexError: list index out of range